<a href="https://colab.research.google.com/github/anushan1989/helpmate_ai/blob/main/helpmate_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pdfplumber tiktoken openai chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:

In [3]:
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
from sentence_transformers import CrossEncoder, util
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb

In [6]:
pdf_path='/content/Principal-Sample-Life-Insurance-Policy.pdf'
with pdfplumber.open(pdf_path) as pdf:

    single_page = pdf.pages[6]
    text = single_page.extract_text()
    tables = single_page.extract_tables()
    print(text)

Section A – Eligibility
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Section B - Effective Dates
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Section C - Individual Terminations
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Termination for Fraud Article 4
Coverage While Outside of the United States Article 5
Section D - Continuation
Member Life Insurance Article 1
Dependent Insurance - Developmentally Disabled or
Physically Handicapped Children Article 2
Section E - Reinstatement
Reinstatement Article 1
Federal Required Family and Medical Leave Act (FMLA) Article 2
Reinstatement of Coverage for a Member or Dependent When
Coverage Ends due to Living Outside of the United States Article 3
Section F - Individual Purchase Rights
Member Life In

In [7]:
def check_bboxes(word, bbox):
    """
    Helper function to check if a word's bbox is within a table's bbox.
    """
    word_bbox = (word['x0'], word['top'], word['x1'], word['bottom'])
    return (word_bbox[0] >= bbox[0] and word_bbox[2] <= bbox[2] and
            word_bbox[1] >= bbox[1] and word_bbox[3] <= bbox[3])

def extract_text_from_pdf(pdf_path):
    full_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for p, page in enumerate(pdf.pages, start=1):
            page_no = f"Page {p}"
            text = page.extract_text()

            # Extract heading (if text exists)
            heading = text.split('\n')[0].strip() if text else None

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                check_bboxes(word, table_bbox) for table_bbox in table_bboxes)]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):
                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass
                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))

            full_text.append([page_no, heading, " ".join(lines)])

    # Convert the extracted data to a DataFrame
    df = pd.DataFrame(full_text, columns=['Page_Number', 'Heading', 'Text'])

    return df

In [8]:
df = extract_text_from_pdf(pdf_path)

In [9]:
df.head(10)

,Page_Number,Heading,Text
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...
1,Page 2,This page left blank intentionally,This page left blank intentionally
2,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...
3,Page 4,This page left blank intentionally,This page left blank intentionally
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...
5,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...
6,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...
7,Page 8,Section A - Member Life Insurance,Section A - Member Life Insurance Schedule of ...
8,Page 9,P ART I - DEFINITIONS,P ART I - DEFINITIONS When used in this Group ...
9,Page 10,T he legally recognized union of two eligible ...,T he legally recognized union of two eligible ...


In [10]:
df['Text'][9]

'T he legally recognized union of two eligible individuals of the same sex established according to law. Civil Union Partner For two persons to establish a Civil Union in Rhode Island, it shall be necessary that they satisfy all of the following criteria: a. not be a party to another Civil Union or marriage in Rhode Island; b. be of the same sex and therefore be excluded from the marriage laws of Rhode Island or any other state; c. be at least 18 years of age; d. not be related to the other proposed party to the Civil Union. NOTE: For the purposes of this Group Policy, the term "spouse" will include Civil Union Partner, except as otherwise provided in this Group Policy. Date of Issue The date this Group Policy is placed in force: November 1, 2007. Dependent a. A Member\'s spouse, if that spouse: (1) is legally married to the Member; and (2) is not in the Armed Forces of any country; and (3) is not insured under this Group Policy as a Member. A Member\'s spouse will also include a Civil

In [11]:
df['Text_Length'] = df['Text'].apply(lambda x: len(x.split(' ')))

In [12]:
df.head(10)

,Page_Number,Heading,Text,Text_Length
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30
1,Page 2,This page left blank intentionally,This page left blank intentionally,5
2,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230
3,Page 4,This page left blank intentionally,This page left blank intentionally,5
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110
5,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153
6,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...,176
7,Page 8,Section A - Member Life Insurance,Section A - Member Life Insurance Schedule of ...,171
8,Page 9,P ART I - DEFINITIONS,P ART I - DEFINITIONS When used in this Group ...,387
9,Page 10,T he legally recognized union of two eligible ...,T he legally recognized union of two eligible ...,251


In [13]:
df = df.loc[df['Text_Length']>=10]

In [14]:
df.head(10)

,Page_Number,Heading,Text,Text_Length
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30
2,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110
5,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153
6,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...,176
7,Page 8,Section A - Member Life Insurance,Section A - Member Life Insurance Schedule of ...,171
8,Page 9,P ART I - DEFINITIONS,P ART I - DEFINITIONS When used in this Group ...,387
9,Page 10,T he legally recognized union of two eligible ...,T he legally recognized union of two eligible ...,251
10,Page 11,(2) has been placed with the Member or spouse ...,(2) has been placed with the Member or spouse ...,299
11,Page 12,An institution that is licensed as a Hospital ...,An institution that is licensed as a Hospital ...,352


In [15]:
df.head()

,Page_Number,Heading,Text,Text_Length
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30
2,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110
5,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153
6,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...,176


In [16]:
df['Metadata'] = df.apply(
    lambda x: {
        'Section': (x['Heading'][:20] if x['Heading'] else ''),
        'Page_No.': x['Page_Number']
    },
    axis=1
)

In [17]:
def chunk_text(text, chunk_size=300, overlap_size=50):
    # Split the text into individual words
    words = text.split()
    chunks = []

    # Iterate over the words to create chunks with overlap
    for i in range(0, len(words), chunk_size - overlap_size):
        # Create a chunk from the current position
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)

    return chunks


In [18]:
df['Chunks'] = df['Text'].apply(lambda x: chunk_text(x))

# Flatten the DataFrame to have one row per chunk
chunked_df = df.explode('Chunks').reset_index(drop=True)

# Add an identifier to each chunk to keep track of the page and chunk number
chunked_df['Chunk_ID'] = chunked_df.index + 1

In [19]:
chunked_df.head(20)

,Page_Number,Heading,Text,Text_Length,Metadata,Chunks,Chunk_ID
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30,"{'Section': 'DOROTHEA GLAUSE S655', 'Page_No.'...",DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,1
1,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230,"{'Section': 'POLICY RIDER', 'Page_No.': 'Page 3'}",POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,2
2,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110,"{'Section': 'PRINCIPAL LIFE INSUR', 'Page_No.'...",PRINCIPAL LIFE INSURANCE COMPANY (called The P...,3
3,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153,"{'Section': 'TABLE OF CONTENTS', 'Page_No.': '...",TABLE OF CONTENTS PART I - DEFINITIONS PART II...,4
4,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...,176,"{'Section': 'Section A – Eligibil', 'Page_No.'...",Section A – Eligibility Member Life Insurance ...,5
5,Page 8,Section A - Member Life Insurance,Section A - Member Life Insurance Schedule of ...,171,"{'Section': 'Section A - Member L', 'Page_No.'...",Section A - Member Life Insurance Schedule of ...,6
6,Page 9,P ART I - DEFINITIONS,P ART I - DEFINITIONS When used in this Group ...,387,"{'Section': 'P ART I - DEFINITION', 'Page_No.'...",P ART I - DEFINITIONS When used in this Group ...,7
7,Page 9,P ART I - DEFINITIONS,P ART I - DEFINITIONS When used in this Group ...,387,"{'Section': 'P ART I - DEFINITION', 'Page_No.'...",f. Continence - the ability to voluntarily con...,8
8,Page 10,T he legally recognized union of two eligible ...,T he legally recognized union of two eligible ...,251,"{'Section': 'T he legally recogni', 'Page_No.'...",T he legally recognized union of two eligible ...,9
9,Page 10,T he legally recognized union of two eligible ...,T he legally recognized union of two eligible ...,251,"{'Section': 'T he legally recogni', 'Page_No.'...",2,10


In [20]:
# chunked_df['Metadata'] = chunked_df.apply(
#     lambda x: {
#         'Section': (x['Heading'][:20] if x['Heading'] else ''),
#         'Page_No.': x['Page_Number'],
#         'Chunk_ID':x['Chunk_ID']
#     },
#     axis=1
# )

In [21]:
filepath = "/content/OPEN_AI_KEY.txt"

with open(filepath , "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [22]:
chroma_data_path = '/kaggle/working/'
client = chromadb.PersistentClient(path=chroma_data_path)

In [23]:
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name='text-embedding-ada-002')
insurance_collection = client.get_or_create_collection(name='InsurancePolicyDoc', embedding_function=embedding_function)

In [24]:
documents_list = chunked_df["Chunks"].tolist()
metadata_list = chunked_df['Metadata'].tolist()

In [25]:
insurance_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)